### Define the system prompt

The system prompt defines your agent’s role and behavior. Keep it specific and actionable:

In [1]:
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

### Create tools

**Tools** let a model interact with external systems by calling functions you define. <br>
Tools can depend on **runtime context** and also interact with **agent memory**.<br>
Notice below how the ```python get_user_location``` tool uses runtime context:

In [ ]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"


# dataclass
# Python 标准库
# 用来快速定义只包含数据的类
# 自动生成 __init__, __repr__ 等方法
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# ToolRuntime
# LangChain 中的 运行时容器
# 用来在 tool 执行时：
# 访问上下文（context）
# 访问当前会话信息、用户信息等

# 这里用的是：
# runtime: ToolRuntime[Context]

# 意味着：
# 这个 Tool 不从 LLM 输入参数中拿数据
# 而是 从系统运行时上下文中拿数据

# get_user_location 并不是由模型主动获取用户 ID，而是 LangChain 在执行工具时，通过 ToolRuntime 将预先绑定的运行时上下文（Context）注入到工具函数中。
@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

**ToolRuntime 是什么？**

ToolRuntime[Context] 包含什么？
```text
runtime
 ├── context   ← 你定义的 Context 实例
 ├── metadata
 ├── config
 └── other runtime info
```

你这里用到了：
```python
runtime.context.user_id
```

### Configure your model

Set up your **language model** with the right parameters for your use case:

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "openai:gpt-4o",
    # temperature 控制模型“敢不敢随机”
    # 低 → 更保守、更确定、更像“考试答案”
    # 高 → 更发散、更有创造性、更像“头脑风暴”
    temperature=0.5,
    timeout=10,
    max_tokens=1000
)

### Define response format

Optionally, define a structured response format if you need the agent responses to match a specific schema.

In [ ]:
from dataclasses import dataclass

# We use a dataclass here, but Pydantic models are also supported.
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    # | 部分            | 含义          |
    # | ------------- | ----------- |
    # | `str \| None` | 要么是字符串，要么是空 |
    # | `= None`      | 默认值是 None   |
    # | 是否必填          | ❌ 否         |
    weather_conditions: str | None = None

### Add memory

Add **memory** to your agent to maintain state across interactions. This allows the agent to remember previous conversations and context.

In [5]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

### Create and run the agent

Now assemble your agent with all the components and run it!

In [ ]:
from langchain.agents.structured_output import ToolStrategy
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    # 作用：
    # 让 Agent “记住自己上一次对话执行到哪”
    # 它和下面的 thread_id 配合使用。
    checkpointer=checkpointer
)

# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}
# 它是在给 agent.invoke(..., config=config) 传一个运行配置，其中最重要的是：
# thread_id = 用来标识“这一条对话线程/会话”的唯一 ID。
# 你可以把它理解成：同一条聊天的会话编号。

# 为什么需要 thread_id？
# 因为你启用了：

# checkpointer=checkpointer   # 例如 InMemorySaver()

# 有了 checkpointer 之后，框架会在每次调用 agent 时：
# 用 thread_id 去 checkpointer 里查有没有旧的检查点（checkpoint / state）
# 如果有：恢复上次的 state（包括执行进度、历史消息等，取决于实现）
# 执行本轮
# 把更新后的 state 再保存回 thread_id 对应的槽位

# 所以：
# thread_id 就是 “checkpoint 的 key”。
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

ResponseFormat(punny_response="Looks like the sun is in a committed relationship with Florida today! It's sunny and bright, so don't forget your shades.", weather_conditions='Sunny')
ResponseFormat(punny_response="No problem, I'm always here to weather any question you have!", weather_conditions=None)
